In [ ]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Literal, Annotated
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
from langgraph.checkpoint.memory import MemorySaver
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
llm = ChatGroq(model="llama-3.1-8b-instant", api_key=groq_api_key)

In [ ]:
def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']

    # send to llm
    response = llm.invoke(messages).content

    # response store state
    return {'messages': [AIMessage(content=response)]}

In [ ]:
checkpointer = MemorySaver()

graph = StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile()

chatbot = graph.compile(checkpointer=checkpointer)

In [ ]:
chatbot

In [ ]:
thread_id = "1"

while True:

    user_message = input("Type here : ")

    print(user_message)

    if user_message.strip().lower() in ["exit", "quit", "bye"]:
        break

    config = {"configurable" :  {"thread_id" : thread_id}}

    response = chatbot.invoke({"messages" : [HumanMessage(content = user_message)]}, config = config)
    # response = chatbot.invoke(initial_state)['messages'][-1].content[0]["text"]
    print("AI : ", response['messages'][-1].content)

In [ ]:
print(chatbot.get_state(config = config))

In [ ]:
for i in chatbot.get_state(config = config).values["messages"]:
    print(i.content)